In [ ]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import psycopg2
import datetime

In [62]:
dbname = "kddi_18_param"
npyname = "trkn_18"
datedir = "230120"
sttime = "2022-04-01 18:00:00"

In [63]:
#解析結果の読み込み
timeseries_sep = np.load("./" + datedir+'/'+ npyname + ".npy")

In [64]:
timeseries_sep.shape

(24, 207)

In [65]:
timeseries_sep

array([[ 88,  62,  95, ..., 109, 109,   1],
       [ 77,  74,  90, ..., 116, 116,   0],
       [ 64,  74,  86, ..., 106, 106,   0],
       ...,
       [ 13,   4,  24, ...,  26,  26,   0],
       [ 21,  12,  12, ...,  28,  28,   0],
       [ 16,   6,  11, ...,  23,  23,   0]], dtype=int64)

In [66]:
#メッシュ情報の読み込み
csv_file = r'./'+datedir+'/'+ npyname+'_header.csv'
df = pd.read_csv(csv_file, dtype={'date':str}, index_col=0, parse_dates=True)
df_i = df.loc["gridcode"] #gridcode #end_mesh_area

In [67]:
df_i.values.shape

(207,)

In [82]:
df_i = df_i.iloc[::-1]

In [83]:
df_i

traffic.206    5.132077e+10
traffic.205    5.132076e+10
traffic.204    5.132063e+10
traffic.203    5.132063e+10
traffic.202    5.032769e+10
                   ...     
traffic.4      5.032459e+10
traffic.3      5.032454e+10
traffic.2      5.032454e+10
traffic.1      5.032453e+10
traffic        5.032453e+10
Name: gridcode, Length: 207, dtype: float64

In [68]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [ ]:
cur =connection.cursor()
curs =connection.cursor()
meshcnt = 0
for mesh in df_i.values:
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')

    sql = "select times, tbran from " + dbname + " where mesh = "+ str(mesh) + " order by times, mesh;"
    print(str(mesh))
    curs.execute(sql)
    meshlists = curs.fetchall()
    for i in range(timeseries_sep.shape[0]):
        #負数は対象としない
        if timeseries_sep[i][meshcnt] < 0:
            dte = dte + datetime.timedelta(minutes=15)
            print("minus continue:")
            continue
        finds = 0
        for doc in meshlists:
            if doc[0] == dte:
                finds = 1
                if doc[1] != timeseries_sep[i][meshcnt]:
                    print(str(doc[1]) + ":" + str(timeseries_sep[i][meshcnt]))
                    finds = 2
        if finds == 1:
            dte = dte + datetime.timedelta(minutes=15)
            continue
                    
        if finds == 0:
            sql = "INSERT INTO " + dbname + " (times, mesh,tbran) VALUES ('"+ str(dte) + "',"  + str(mesh) + "," + str(timeseries_sep[i][meshcnt]) + ");"
        else:
            sql = "UPDATE " + dbname + " set tbran = "+ str(timeseries_sep[i][meshcnt]) + " where times = '"  + str(dte) + "' and mesh = " + str(mesh) + ";"
        #print(sql)
        cur.execute(sql)
        dte = dte + datetime.timedelta(minutes=15)
    connection.commit()
    meshcnt += 1
cur.close()
curs.close()
connection.commit()

In [88]:
connection.close()

In [ ]:
#ConvertMeshtoRoad